In [ ]:
import torch
import torch.nn as nn
import math
class _Residual_Block(nn.Module):
    def __init__(self):
        super(_Residual_Block, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.in1 = nn.InstanceNorm2d(64, affine=True)
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.in2 = nn.InstanceNorm2d(64, affine=True)

    def forward(self, x):
        identity_data = x
        output = self.relu(self.in1(self.conv1(x)))
        output = self.in2(self.conv2(output))
        output = torch.add(output,identity_data)
        return output 

class _NetG(nn.Module):
    def __init__(self):
        super(_NetG, self).__init__()

        self.conv_input = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=9, stride=1, padding=4, bias=False)
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        
        self.residual = self.make_layer(_Residual_Block, 16)

        self.conv_mid = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn_mid = nn.InstanceNorm2d(64, affine=True)

        self.upscale4x = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.conv_output = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=9, stride=1, padding=4, bias=False)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()

    def make_layer(self, block, num_of_layer):
        layers = []
        for _ in range(num_of_layer):
            layers.append(block())
        return nn.Sequential(*layers)

    def forward(self, x):
        print(x.shape)
        out = self.relu(self.conv_input(x))
        residual = out
        out = self.residual(out)
        out = self.bn_mid(self.conv_mid(out))
        out = torch.add(out,residual)
        out = self.upscale4x(out)
        out = self.conv_output(out)
        return out
    
model = _NetG()
model = model.cuda()

x = torch.zeros([1,3,256,256]).cuda()
output = model(x)

print(output.shape)

In [ ]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F

class _Residual_Block(nn.Module):
    def __init__(self):
        super(_Residual_Block, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.in1 = nn.InstanceNorm2d(64, affine=True)
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.in2 = nn.InstanceNorm2d(64, affine=True)

    def forward(self, x):
        identity_data = x
        output = self.relu(self.in1(self.conv1(x)))
        output = self.in2(self.conv2(output))
        output = torch.add(output,identity_data)
        return output 

class myUpsample(nn.Module):
    def __init__(self, scale_factor, mode='bilinear', align_corners=False):
        self.scale_factor = scale_factor
        self.mode = mode
        self.align_corners = align_corners
    def forward(self, input):
        return F.interpolate(input, scale_factor=self.scale_factor, mode=self.mode, align_corners=self.align_corners)

class NetARW(nn.Module):
    def __init__(self, up_ratio=2, up_type='subpixel'):
        super(NetARW, self).__init__()
        
        self.up_ratio = up_ratio
        self.conv_input = nn.Conv2d(in_channels=4, out_channels=64, kernel_size=9, stride=1, padding=4, bias=False)
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        
        self.residual = self.make_layer(_Residual_Block, 16)

        self.conv_mid = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn_mid = nn.InstanceNorm2d(64, affine=True)
        if up_type == 'subpixel':
            self.upscale2x = nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
                nn.PixelShuffle(2),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
                nn.PixelShuffle(2),
                nn.LeakyReLU(0.2, inplace=True),
            )
            self.upscale2x_1 = nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
                nn.PixelShuffle(2),
                nn.LeakyReLU(0.2, inplace=True),
            )
            self.upscale2x_2 = nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
                nn.PixelShuffle(2),
                nn.LeakyReLU(0.2, inplace=True),
            )
        if up_type == 'deconv':
            self.upscale2x = nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
                nn.ConvTranspose2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
                #F.interpolate(g_, scale_factor=self.up_factor, mode='bilinear', align_corners=False)
                nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
                nn.ConvTranspose2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
                nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
                nn.LeakyReLU(0.2, inplace=True),
            )
            self.upscale2x_1 = nn.Sequential(
                nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
                nn.ConvTranspose2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
                nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
                nn.LeakyReLU(0.2, inplace=True),
            )
            self.upscale2x_2 = nn.Sequential(
                nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
                nn.ConvTranspose2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
                nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
                nn.LeakyReLU(0.2, inplace=True),
            )
        
        self.conv_output = nn.Conv2d(in_channels=256, out_channels=3, kernel_size=9, stride=1, padding=4, bias=False)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()

    def make_layer(self, block, num_of_layer):
        layers = []
        for _ in range(num_of_layer):
            layers.append(block())
        return nn.Sequential(*layers)

    def forward(self, x):
        print(x.shape)
        out = self.relu(self.conv_input(x))
        residual = out
        out = self.residual(out)
        out = self.bn_mid(self.conv_mid(out))
        out = torch.add(out,residual)
        out = self.upscale2x(out)
        if self.up_ratio == 4:
            out = self.upscale2x_1(out)
        
        if self.up_ratio == 8:
            out = self.upscale2x_1(out)
            out = self.upscale2x_2(out)
        
        print(out.shape)
        out = self.conv_output(out)
        return out

model = NetARW(up_type="deconv", up_ratio=2)
model = model.cuda()
x = torch.zeros([1,4,64,64]).cuda()
output = model(x)

print(output.shape)

In [ ]:
import torch
import numpy as np
a = torch.randn(1, 3, 256, 256)
#a = torch.from_numpy(np.arange(0,256)).view(1,1,16,16)
#print(a)
print(a.shape)
kernel_size = 8
kernel_stride = 8
a = a.unfold(2, kernel_size, kernel_stride)
print(a.shape)

a = a.unfold(3, kernel_size, kernel_stride)
print(a.shape)
a = a.contiguous().view(a.size(0), a.size(1), -1, a.size(4), a.size(5))
print(a.shape)
# #print(a)

# a.permute((0,2,3,4,1)).shape
a = a.view(a.shape[:5])
print(a.shape)



In [3]:
import torch.nn as nn
import torch
class PatchNonlocalPool(nn.Module):

    def __init__(self, n_feats, patch_size):
        super(PatchNonlocalPool,self).__init__()
        self.patch_size = patch_size
        #self.conv = nn.Conv2d(1024, 1024,kernel_size=patch_size,stride=1,padding=0,bias=False)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        #self.avg_pool2 = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        b,n,h,w = x.shape
        kernel_size = self.patch_size
        kernel_stride = self.patch_size
        # [b, 64, 32, 32, 8, 8]
        # x [b, 64, 256, 256] => [b, 64, 8, 8, 1024]
        x = x.view(x.shape[0]*x.shape[1], x.shape[2],x.shape[3])
        a = x.unfold(1, kernel_size, kernel_stride).unfold(2,kernel_size,kernel_stride)
        a = a.contiguous().view(a.size(0), -1, a.size(3), a.size(4))
        # => [b*64, 1024, 8, 8]
        #[b*64,1024,64] a_i
        a1 = a.view(*a.shape[:2],-1)
        #[b*64,64,1024] a_j
        a2 = a1.permute((0,2,1))
        #[b*64,1024,1024] => f(x_i, x_j)
        f1 = torch.bmm(a1, a2)
        #print(f1.shape)
        #[b*64,1024,1,1]
        #print(a.shape)
        #[b*64,1024,1,1]
        y1 = self.avg_pool(a)
        #[b*64,1024,1]
        y1 = y1.view(y1.shape[:3])
        
        #[b*64,1024,1024]
        y2 = torch.mul(f1,y1)
        
        y3 = self.avg_pool(y2)
        return y3.view(b,n,1,1)

PNLpool = PatchNonlocalPool(64,16)
PNLpool = PNLpool
#.cuda()
x = torch.randn(16,64,256,256)
#.cuda()
x = PNLpool(x)
print(x.shape)

torch.Size([16, 64, 1, 1])


In [ ]:
a = torch.randn(3,4,10)
b = torch.randn(3,10,4)

torch.matmul(a,b).shape

In [ ]:

h = w = 256
n = 20

print(h % n != 0)
if h % n != 0:
    print("hello world")
if h % n != 0 or w % n != 0 :
    raise ValueError("n: %d can not be division by h: %d or w: %d" %(n, h, w))

In [ ]:
import os
data_root = '/store/dataset/SR/train_data/SRRAW/X4/train/'
file_name = '00579_00007.png'
diff_path = os.path.join(data_root, 'Diff', file_name)
hr_path = os.path.join(data_root, 'HR', file_name)

from PIL import Image
diff_img_o = Image.open(diff_path)
hr_img = Image.open(hr_path)

new_height = new_width = 512
width, height = diff_img_o.size   # Get dimensions

left = (width - new_width)/2
top = (height - new_height)/2
right = (width + new_width)/2
bottom = (height + new_height)/2

diff_img_o = diff_img_o.crop((left, top, right, bottom))
hr_img = hr_img.crop((left, top, right, bottom))
#diff_img = diff_img_o.convert("L")
diff_img = diff_img_o
import numpy as np

diff_img_np = np.array(diff_img)
hr_img_np = np.array(hr_img)

diff_img_np = (diff_img_np > 50).astype(np.float32)

#diff_img_np = diff_img_np[...,np.newaxis]
#diff_img_np = np.broadcast_to(diff_img_np,(new_height,new_width,3))

print(diff_img_np.shape)
print(hr_img_np.shape)
hr_img_np = np.multiply(hr_img_np, diff_img_np).astype(np.uint8)
#print(hr_img_np)



from matplotlib import pyplot as plt
plt.figure(figsize=(16,16))



plt.subplot(221)
plt.imshow(diff_img_np, cmap='gray')

plt.subplot(222)
plt.imshow(hr_img_np)

plt.subplot(223)
plt.imshow(diff_img_o)

plt.subplot(224)
plt.imshow(diff_img_o)

plt.show()

In [10]:
import torch

a = torch.randn(4,3)

print(a)
print(a.shape)

import torch.nn.functional as F
b = F.softmax(a, dim=-1)

print(b)

tensor([[-1.0115,  0.7080,  0.4029],
        [ 1.1801, -0.2080,  0.9660],
        [-0.3408, -0.1401,  0.7713],
        [-0.7147,  0.8593, -0.0528]])
torch.Size([4, 3])
tensor([[0.0935, 0.5219, 0.3846],
        [0.4862, 0.1213, 0.3925],
        [0.1900, 0.2322, 0.5778],
        [0.1288, 0.6216, 0.2497]])


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class PatchNonlocalPool(nn.Module):
    def __init__(self, n_feats, patch_size=16):
        super(PatchNonlocalPool,self).__init__()
        self.patch_size = patch_size
        self.conv = nn.Conv2d(n_feats, n_feats,kernel_size=3,stride=1,padding=1,bias=False)
        self.avg_pool1 = nn.AdaptiveAvgPool2d(1)
        self.avg_pool2 = nn.AdaptiveAvgPool1d(1)

    def forward(self, x):
        b,n,h,w = x.shape
        kernel_size = self.patch_size
        kernel_stride = self.patch_size
        # [b*64, 16, 16, 16, 16]
        # x [b*64, 256, 256] => [b*64, 256, 16, 16]
        x = x.view(b*n, h,w)
        a = x.unfold(1, kernel_size, kernel_stride).unfold(2,kernel_size,kernel_stride)
        a = a.contiguous().view(a.size(0), -1, a.size(3), a.size(4))
        # => [b*64, 256, 16, 16]
        #[b*64,256,256_fm] a_i
        a1 = a.view(*a.shape[:2],-1)
        #[b*64,256_fm,256] a_j
        a2 = a1.permute((0,2,1))
        #[b*64,256,256] => f(x_i, x_j)
        f1 = torch.matmul(a1, a2)
        f_div_C = F.softmax(f1, dim=-1)
        #[b*64,256,1,1]
        y1 = self.avg_pool1(a)
        #[b*64,256,1]
        y1 = y1.view(y1.shape[:3])
        #[b*64,256,256]
        #y2 = torch.mul(f1,y1)
        #[b*64,256,1]
        y2 = torch.matmul(f_div_C, y1)
        #y3 = self.avg_pool2(y2)
        #[b, 64, 16, 16]
        y2 = y2.contiguous().view(b,n, int(h/self.patch_size), int(w/self.patch_size))
        y2 = self.conv(y2)
        #[b,64,1,1]
        y2 = self.avg_pool1(y2)
        return y2

a = torch.randn(16,64,256,256)
pp = PatchNonlocalPool(n_feats=64)


tensor([[[[-1.4453e+00,  7.6660e-01, -1.6690e+00,  ..., -3.2500e-01,
           -3.1606e-01, -1.6864e+00],
          [-1.8694e-01,  1.3925e+00, -1.1483e+00,  ...,  8.2272e-02,
           -1.3943e+00, -2.9364e-01],
          [-7.7659e-02,  1.1806e+00,  4.3352e-01,  ..., -9.9556e-01,
            2.9232e-01,  1.5797e+00],
          ...,
          [-3.5963e-01,  9.1163e-01,  7.9232e-01,  ...,  2.4011e-01,
           -4.4147e-01,  2.5068e-01],
          [-3.0067e-01,  2.0332e+00,  1.5137e+00,  ..., -9.6034e-01,
           -5.8280e-01,  3.5312e-01],
          [-9.4486e-01, -1.9564e+00,  1.1405e+00,  ...,  2.1508e-02,
            3.8641e-01,  1.3860e+00]],

         [[ 2.9581e-01, -3.4922e-01,  1.5355e+00,  ...,  3.1994e-01,
            9.6983e-01,  1.1500e+00],
          [-5.6420e-01, -2.2248e+00,  3.0385e-02,  ..., -2.5731e+00,
           -3.3326e+00, -5.3650e-01],
          [-1.0598e+00,  8.1447e-01, -1.3055e+00,  ...,  9.4776e-02,
           -5.0595e-01,  5.4212e-01],
          ...,
     

tensor([[[[-3.9515e-03]],

         [[-1.5727e-03]],

         [[ 3.5080e-03]],

         ...,

         [[-2.1723e-03]],

         [[ 2.6100e-03]],

         [[ 1.5809e-03]]],


        [[[-1.5049e-04]],

         [[ 4.0881e-03]],

         [[ 4.4255e-04]],

         ...,

         [[ 7.4579e-04]],

         [[-3.9191e-03]],

         [[-1.7382e-04]]],


        [[[ 1.8458e-04]],

         [[ 2.8802e-04]],

         [[-3.6983e-03]],

         ...,

         [[ 4.6894e-03]],

         [[-6.4117e-04]],

         [[ 7.8141e-04]]],


        ...,


        [[[-7.1010e-04]],

         [[-2.7668e-03]],

         [[ 7.9627e-04]],

         ...,

         [[ 8.5203e-04]],

         [[ 5.4320e-04]],

         [[ 1.5713e-03]]],


        [[[ 5.4359e-04]],

         [[-8.3872e-06]],

         [[ 8.0392e-04]],

         ...,

         [[ 1.0794e-03]],

         [[ 2.4431e-03]],

         [[-7.9084e-04]]],


        [[[ 1.4515e-03]],

         [[ 5.3454e-04]],

         [[-1.6268e-03]],

         